In [1]:
import tensorflow as tf

2023-08-02 23:48:08.104357: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 23:48:19.464798: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os,timeit
import numpy as np

In [3]:
from numpy import random

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/tyagi/Desktop/wheat/data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\mvl\wheat\data\BULK\\"
    SLASH="\\"

In [6]:
#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.90
TOTAL_IMAGE_COUNT = 2400
IMAGE_COUNT = int(TOTAL_IMAGE_COUNT/4)
NUM_VARIETIES = 4
NUM_OF_BANDS = 15

REMOVE_NOISY_BANDS = False
FIRST_BAND = 15
LAST_BAND = 161

IMAGE_WIDTH = 30
IMAGE_HEIGHT = 30

In [7]:
from enum import Enum

class filter_method(Enum):
    none = 0
    snv = 1
    msc = 2
    savgol = 3
    
FILTER = filter_method(1).name

# to be set if filter chosen is savgol
WINDOW = 7
ORDER = 2
DERIVATIVE = "none"

In [8]:
from enum import Enum
 
class feature_extraction_method(Enum):
    none = 0
    pca_loading = 1
    lda = 2
    ipca = 3

FEATURE_EXTRACTION = feature_extraction_method(0).name

NUM_OF_BANDS = 3
if FEATURE_EXTRACTION == "pca_loading" or FEATURE_EXTRACTION == "ipca":
    NUM_OF_BANDS = 8
elif FEATURE_EXTRACTION == "lda":
    NUM_OF_BANDS = 3
    assert NUM_OF_BANDS <= min(NUM_VARIETIES-1,168),"NUM_OF_BANDS is greater."

In [9]:
# List for All varieties
VARIETIES = []
VARIETIES_CODE = {}

for name in os.listdir(DATA_DIRECTORY):
    if (name.endswith(".hdr") or name.endswith(".bil")):
        continue
    VARIETIES_CODE[name] = len(VARIETIES)
    VARIETIES.append(name)
    if len(VARIETIES)==NUM_VARIETIES:
        break

In [10]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [11]:
def dataset_file_name(variety):
    name = "./dataset/V"+str(variety).zfill(3)+"_IC_"+str(TOTAL_IMAGE_COUNT).zfill(5)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)+"_FILTER_"+str(FILTER)+"_FeatureExtraction_"+str(FEATURE_EXTRACTION)
    if REMOVE_NOISY_BANDS:
        name+="_REMOVE_NOISY_BANDS_"+str(REMOVE_NOISY_BANDS)
    if FILTER == "savgol":
        name+="_WINDOW_"+str(WINDOW)+"_ORDER_"+str(ORDER)
    return name

In [12]:
train_dataset = []
train_dataset_label = []
test_dataset=[]
test_dataset_label = []

for idx, v in enumerate(VARIETIES):
    print("idx: ",idx)
    if idx >= NUM_VARIETIES:
        break
    train_dataset= train_dataset + np.load(dataset_file_name(v)+"_train_dataset.npy").tolist()
    train_dataset_label = train_dataset_label + np.load(dataset_file_name(v)+"_train_dataset_label.npy").tolist()
    test_dataset = test_dataset + np.load(dataset_file_name(v)+"_test_dataset.npy").tolist()
    test_dataset_label = test_dataset_label + np.load(dataset_file_name(v)+"_test_dataset_label.npy").tolist()
    
train_dataset = np.array(train_dataset)
train_dataset_label = np.array(train_dataset_label)
test_dataset = np.array(test_dataset)
test_dataset_label = np.array(test_dataset_label)

idx:  0
idx:  1
idx:  2
idx:  3


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [15]:
nsamples, nx, ny,nb = train_dataset.shape
nsamplestest, nx, ny,nb = test_dataset.shape
d2_train_dataset = train_dataset.reshape((nsamples,nx*ny*nb))
d2_test_dataset = test_dataset.reshape((nsamplestest,nx*ny*nb))

In [16]:
# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 100)

In [17]:
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
tic = start_timer()
clf.fit(d2_train_dataset, train_dataset_label)
toc = end_timer()
show_time(tic,toc)

Testing started
Testing time (s) = 382.864720475



In [18]:
# performing predictions on the test dataset
y_pred = clf.predict(d2_test_dataset)

print("ACCURACY OF THE MODEL: ", accuracy_score(test_dataset_label, y_pred))

ACCURACY OF THE MODEL:  0.6541666666666667


In [19]:
print(confusion_matrix(test_dataset_label, y_pred))

[[286  99  38  57]
 [105 240  55  80]
 [ 22  18 414  26]
 [ 55  72  37 316]]


In [20]:
# creating a RF classifier
clf2 = RandomForestClassifier(n_estimators = 200)

In [21]:
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
tic = start_timer()
clf2.fit(d2_train_dataset, train_dataset_label)
toc = end_timer()
show_time(tic,toc)

Testing started
Testing time (s) = 769.3837535999992



In [22]:
# performing predictions on the test dataset
y_pred = clf2.predict(d2_test_dataset)

print("ACCURACY OF THE MODEL: ", accuracy_score(test_dataset_label, y_pred))

ACCURACY OF THE MODEL:  0.6911458333333333


In [23]:
print(confusion_matrix(test_dataset_label, y_pred))

[[304  74  39  63]
 [ 87 261  53  79]
 [ 17  21 423  19]
 [ 48  65  28 339]]


In [24]:
# creating a RF classifier
clf3 = RandomForestClassifier(n_estimators = 400)

In [25]:
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
tic = start_timer()
clf3.fit(d2_train_dataset, train_dataset_label)
toc = end_timer()
show_time(tic,toc)

Testing started
Testing time (s) = 1550.4307582929996



In [26]:
# performing predictions on the test dataset
y_pred = clf3.predict(d2_test_dataset)

print("ACCURACY OF THE MODEL: ", accuracy_score(test_dataset_label, y_pred))

ACCURACY OF THE MODEL:  0.7286458333333333


In [27]:
print(confusion_matrix(test_dataset_label, y_pred))

[[317  71  33  59]
 [ 75 289  44  72]
 [ 17  12 424  27]
 [ 29  65  17 369]]


In [28]:
for x in range(5,10):
    clf3 = RandomForestClassifier(n_estimators = x*100)
    tic = start_timer()
    clf3.fit(d2_train_dataset, train_dataset_label)
    toc = end_timer()
    show_time(tic,toc)
    
    y_pred = clf3.predict(d2_test_dataset)
    
    print("ACCURACY OF THE MODEL: ", accuracy_score(test_dataset_label, y_pred))
    print(confusion_matrix(test_dataset_label, y_pred))

Testing started
Testing time (s) = 1911.6362738890002

ACCURACY OF THE MODEL:  0.7229166666666667
[[324  70  37  49]
 [ 82 268  55  75]
 [ 16  11 428  25]
 [ 34  57  21 368]]
Testing started
Testing time (s) = 2268.924234922999

ACCURACY OF THE MODEL:  0.7276041666666667
[[333  58  40  49]
 [ 82 269  57  72]
 [ 15  13 430  22]
 [ 32  66  17 365]]
Testing started
Testing time (s) = 2648.54316913

ACCURACY OF THE MODEL:  0.725
[[327  65  39  49]
 [ 78 278  52  72]
 [ 16   9 428  27]
 [ 42  57  22 359]]
Testing started
Testing time (s) = 3032.4836687530005

ACCURACY OF THE MODEL:  0.7244791666666667
[[327  55  39  59]
 [ 84 267  58  71]
 [ 16   9 430  25]
 [ 30  67  16 367]]
Testing started
Testing time (s) = 3427.0532836310012

ACCURACY OF THE MODEL:  0.7359375
[[330  68  37  45]
 [ 71 282  54  73]
 [ 18   9 425  28]
 [ 26  61  17 376]]


In [29]:
for x in range(10,15):
    clf3 = RandomForestClassifier(n_estimators = x*100)
    tic = start_timer()
    clf3.fit(d2_train_dataset, train_dataset_label)
    toc = end_timer()
    show_time(tic,toc)
    
    y_pred = clf3.predict(d2_test_dataset)
    
    print("Estimators: ",x*100,"  ACCURACY OF THE MODEL: ", accuracy_score(test_dataset_label, y_pred))
    print(confusion_matrix(test_dataset_label, y_pred))

Testing started
Testing time (s) = 3918.6789783580025

Estimators:  1000   ACCURACY OF THE MODEL:  0.7265625
[[327  55  40  58]
 [ 83 274  52  71]
 [ 18  10 426  26]
 [ 30  63  19 368]]
Testing started
Testing time (s) = 4159.905602744002

Estimators:  1100   ACCURACY OF THE MODEL:  0.7338541666666667
[[332  61  38  49]
 [ 79 280  54  67]
 [ 13  12 429  26]
 [ 35  57  20 368]]
Testing started
Testing time (s) = 4541.535053742999

Estimators:  1200   ACCURACY OF THE MODEL:  0.7255208333333333
[[327  59  39  55]
 [ 78 270  56  76]
 [ 14  11 431  24]
 [ 32  60  23 365]]
Testing started
Testing time (s) = 4923.764090279998

Estimators:  1300   ACCURACY OF THE MODEL:  0.7364583333333333
[[336  56  37  51]
 [ 82 273  53  72]
 [ 14  10 434  22]
 [ 23  64  22 371]]
Testing started
Testing time (s) = 5294.585970604006

Estimators:  1400   ACCURACY OF THE MODEL:  0.7296875
[[338  55  33  54]
 [ 86 265  55  74]
 [ 15   9 429  27]
 [ 32  57  22 369]]
